# Linux Server Administration - Firewalls

#### 1. Topics/Sub-Topics

- Good firewall policy has 3 primary considerations:
    - Management Rules
    - Access Rules
    - Default Deny Rule
- As well as recurring security themes:
    - Priniciple of Least Privilege/Deny By Default
    - Defense in Depth
    - Keep it simple
        - Advanced rules and ordering can be broken
    - Logging, logging, logging
    - TEST, TEST, TEST
- Levels of Defense
    - Ensure gateway firewalls reinforce local policies
- Traffic Types
    - Ingress
    - Egress
- iptables
    - Tool to control `netfilter` which is the underlying technology
    - Stateful Packet Inspection(SPI)
    - Order matters
    - Basic syntax:
        - `sudo iptables -A <category> -p <protocol> -j <action>`
        - Tables
            - FILTER - Standard processing of packets
            - NAT - Rewrite source/destination of packets or track sessions
            - MANGLE - Modify portions of the packet
        - Chains(aka Categories)
            - INPUT(ingress)
            - OUTPUT(egress)
            - FORWARD(traffic forwarded from one interface to another)
        - Table/Chain Combinations
            - FILTER: Input, Output, Forward
            - NAT: Prerouting, Postrouting, Output
            - MANGLE: Prerouting, Postrouting, Input, Output, Forward
        - Timing
            - PREROUTING: Immediately after being received by an interface.
            - POSTROUTING: Right before leaving an interface.
            - INPUT: Right before being handed to a local process.
            - OUTPUT: Right after being created by a local process.
            - FORWARD: For any packets coming in one interface and leaving out another.
        - Actions(aka Targets)
            - ACCEPT - Let packet through
            - REJECT - Deny packet
            - DROP - "drop it on the floor"
            - LOG - Log to syslog
        - Filters
            - Source/Destination
                - IP address(es)/subnets
            - Protocol
        - Persistence
            - `iptables-save`
            - `iptables-restore`
            - Current rules only apply to current running session if not saved
            - Save current rules: 
                - `sudo iptables-save > /etc/sysconfig/iptables` <- Red Hat
                - `sudo iptables-save > /etc/iptables/rules.v4` <- Debian
            - Restore current rules(manually):
                - `sudo iptables-restore < /etc/sysconfig/iptables` <- Red Hat
                - `sudo iptables-restore < /etc/iptables/rules.v4` <- Debian
            - IMPORTANT: New rules must be *RESAVED*
    - Examples:
        - Add Rule: `sudo iptables -A INPUT -p icmp -j ACCEPT`
        - Delete Rule: `sudo iptables -D INPUT - p icmp -j ACCEPT`
        - Flush Chain: `sudo iptables -F INPUT`
        - Flush ALL Chains: `sudo iptables -F`
        - Allow Port: `sudo iptables -A INPUT -m tcp -p tcp --dport 22 -j ACCEPT`
        - Allow Source: `sudo iptables -A INPUT -m tcp -p tcp -s 10.0.0.0/255.255.255.0 --dport 22 -j ACCEPT`
        - Allow Destination: `sudo iptables -A OUTPUT -p icmp -d 10.0.0.0/255.255.255.0 -j ACCEPT`
        - Allow interface: `sudo iptables -A INPUT -i lo -j ACCEPT`
        - Allow stateful outgoing web traffic: 
          ```
          sudo iptables -A INPUT -o eth0 -p TCP –dport 80 -j ACCEPT 
          sudo iptables -A INPUT -i eth0 -p TCP -m state –state ESTABLISHED,RELATED –sport 80 -j ACCEPT
           ```
        - NAT Forwarding: 
            ```
            sudo iptables -t nat -A PREROUTING -i eth0 -p tcp -d 1.2.3.4 –dport 25 -j DNAT –to 192.168.0.2:25
            sudo iptables -A FORWARD -i eth0 -o eth1 -p tcp –dport 25 -d 192.168.0.2 -j ACCEPT
            ```
        - Setup Logging(order matters):
            ```
            sudo iptables -A INPUT -p tcp --dport 80 -j LOG --log-level 7 --log-prefix "Accept HTTP"
            sudo iptables -A INPUT -d 138.23.23.23 -p tcp --dport 80 -j ACCEPT
            ```
         - Turn on remote logging in `/etc/rsyslog.conf`:
             - `kern.debug /var/log/firewall.log`

- ufw aka Firewalls for Dummies
    - To enable:
        `sudo ufw enable`
    - To check rules:
        `sudo ufw status`
    - To disable:
        `sudo ufw disable`
    - To add basic ingress/egress tcp rule:
        `sudo ufw allow http`
        `sudo ufw allow 80`
    - To allow ingress/egress by protocol on a port:
        `sudo ufw allow proto tcp to any port 22`
        `sudo ufw allow 1725/udp`
    - To deny ingress/egress traffic from a specific IP:
        `sudo ufw deny from 15.15.15.51`
    - Deny ingress traffic to a certain interface:
        `sudo ufw deny in on eth0 from 15.15.15.51`
    - Allow ingress/egress traffic from subnet to specific port:
        `sudo ufw allow from 15.15.15.0/24  to any port 22`
    - Allow ingress traffic on a certain interface to a port:
        `sudo ufw allow in on eth1 to any port 3306`
    - Block egress traffic on a given port:
        `sudo ufw deny out 25`
    - Show applications available
        `sudo ufw app list`
    - Allow traffic to a specific application
        `sudo ufw allow from 192.168.0.0/16 to any app Bonjour`
    - Turn on/change logging level(`/var/log/ufw`)
        `sudo ufw logging on`
        `sudo ufw logging high`
        
 
#### 2. Pre-requisites
- Basic Linux knowledge
- Basic networking knowledge of types of protocols and common ports

#### 3. Desired Outcomes
- Students should understand the basic purpose of firewalls

#### 4. Resources
- https://danielmiessler.com/study/iptables/
- https://danielmiessler.com/blog/professional-firewall-iptables/
- https://www.netfilter.org/
- https://www.digitalocean.com/community/tutorials/ufw-essentials-common-firewall-rules-and-commands
- http://manpages.ubuntu.com/manpages/precise/man8/ufw.8.html

#### 5. Challenges/Exercises
- Create a script to setup a basic web server, including the ability to remotely access the server and check on it's basic health
- Create a script to setup a basic DNS server, including the ability to remotely access the server and check on it's basic health
- Set up a local private interface and NAT traffic to that internal port
- Install `fail2ban` and direct traffic to the site and inspect change in firewall rules
- Write a `fail2ban` mini clone
```
fail2ban is, as you say, essentially just a script that does regex on log files, extracts malicious sources from those logs (e.g. failed SSH login, web client triggering repeated 4xx or 5xx respones, or having a user-agent associated with known attacker profiles). It integrates with iptables (and whatever might be running on top of iptables, e.g. firewalld, shorewall), in order to block traffic related to malicious hosts. Those blocks tend to be implemented either as simple iptables rules, or as an iptables rule + an ipset.
```

#### 6. Self Study Objectives
- Read both Daniel Miessler links in and out
- Make sure any servers you run are setup with good policy

#### 7. Estimated Time

#### 8. Applicable Standards